In [1]:
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.static import players
import pandas as pd

In [2]:
import time, random
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def nba_session():
    s = requests.Session()

    # These headers matter for stats.nba.com
    s.headers.update({
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122 Safari/537.36",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.9",
        "Origin": "https://www.nba.com",
        "Referer": "https://www.nba.com/",
        "Connection": "keep-alive",
    })

    retry = Retry(
        total=8,
        connect=8,
        read=8,
        backoff_factor=1.2,              # exponential-ish backoff
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["GET", "POST"],
        raise_on_status=False,
        respect_retry_after_header=True,
    )

    adapter = HTTPAdapter(max_retries=retry, pool_connections=50, pool_maxsize=50)
    s.mount("https://", adapter)
    s.mount("http://", adapter)
    return s

SESSION = nba_session()

def polite_sleep(min_s=0.6, max_s=1.6):
    time.sleep(random.uniform(min_s, max_s))


# Imported Libraries and NBA API
### Import required NBA API modules and core Python libraries for data extraction and processing.

In [3]:
import pandas as pd
from nba_api.stats.endpoints import leaguegamefinder

def get_finals_game_ids_for_player(player_id: int, season: str) -> list[str]:
    """
    Returns Finals game IDs for a player in a given season (e.g., '2016-17').
    Finals games have GAME_ID containing '040' in the series code.
    """
    gf = leaguegamefinder.LeagueGameFinder(
        player_id_nullable=player_id,
        season_nullable=season,
        season_type_nullable="Playoffs"
    )
    games = gf.get_data_frames()[0]  # game log

    # Finals filter: Finals series games have '040' in the GAME_ID (e.g., 0041600401)
    games["GAME_ID"] = games["GAME_ID"].astype(str)
    finals_games = games[games["GAME_ID"].str.contains("040")].copy()

    return finals_games["GAME_ID"].tolist()


# Player Identification 
### Identify the unique NBA player ID needed to query shot-level data from the NBA Stats API.

In [4]:
from nba_api.stats.endpoints import shotchartdetail

def get_player_shots_for_game(player_id: int, season: str, game_id: str) -> pd.DataFrame:
    scd = shotchartdetail.ShotChartDetail(
        team_id=0,
        player_id=player_id,
        season_nullable=season,
        season_type_all_star="Playoffs",
        game_id_nullable=game_id,
        context_measure_simple="FGA"
    )
    df = scd.get_data_frames()[0]
    return df


# Finals Game Identification 
### Retrieve all NBA Finals game IDs for a given player and season by filtering playoff games to the Finals round.

In [5]:
import time

def finals_shots_last_10_seasons(player_name: str, start_year=2014, end_year=2024):
    """
    Pulls ALL shots taken by player in ALL NBA Finals games from seasons start_year-end_year.
    Seasons are strings like '2016-17'. end_year refers to the ending year of the season.
    """
    from nba_api.stats.static import players

    # find player_id
    p = players.find_players_by_full_name(player_name)[0]
    player_id = p["id"]

    all_shots = []
    summary = []

    for end in range(start_year+1, end_year+1):  # 2016..2024 ends
        season = f"{end-1}-{str(end)[-2:]}"      # '2015-16' style

        finals_game_ids = get_finals_game_ids_for_player(player_id, season)
        summary.append({"season": season, "finals_games_found": len(finals_game_ids)})

        for gid in finals_game_ids:
            df_game = get_player_shots_for_game(player_id, season, gid)
            if not df_game.empty:
                all_shots.append(df_game)
            time.sleep(0.8)  # slow down to avoid blocking

    shots_df = pd.concat(all_shots, ignore_index=True) if all_shots else pd.DataFrame()
    summary_df = pd.DataFrame(summary)

    return shots_df, summary_df

# Shot Data Extraction (Per Game)

# Star Players 
### (LeBron James, Stephen Curry, Giannis Antetokounmpo, Nikola Jokic, Jimmy Butler)

# LeBron James - Cleveland Caveliers and Los Angeles Lakers (2015, 2016, 2017, 2018, 2020)

In [8]:
shots_df, finals_summary = finals_shots_last_10_seasons("LeBron James", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   6
1  2015-16                   7
2  2016-17                   5
3  2017-18                   4
4  2018-19                   0
5  2019-20                   6
6  2020-21                   0
7  2021-22                   0
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 687


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0041400406,16,2544,LeBron James,1610612739,Cleveland Cavaliers,1,10,0,...,Left Side(L),24+ ft.,24,-228,77,1,0,20150616,CLE,GSW
1,Shot Chart Detail,0041400406,19,2544,LeBron James,1610612739,Cleveland Cavaliers,1,9,31,...,Center(C),Less Than 8 ft.,0,2,6,1,1,20150616,CLE,GSW
2,Shot Chart Detail,0041400406,57,2544,LeBron James,1610612739,Cleveland Cavaliers,1,4,50,...,Center(C),Less Than 8 ft.,0,6,0,1,0,20150616,CLE,GSW
3,Shot Chart Detail,0041400406,76,2544,LeBron James,1610612739,Cleveland Cavaliers,1,2,43,...,Center(C),Less Than 8 ft.,1,-11,7,1,0,20150616,CLE,GSW
4,Shot Chart Detail,0041400406,79,2544,LeBron James,1610612739,Cleveland Cavaliers,1,2,20,...,Center(C),Less Than 8 ft.,0,9,-3,1,1,20150616,CLE,GSW


# Stephen Curry - Golden State Warriors (2015, 2016, 2017, 2018, 2019, 2022)

In [11]:
shots_df, finals_summary = finals_shots_last_10_seasons("Stephen Curry", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df

    season  finals_games_found
0  2014-15                   6
1  2015-16                   7
2  2016-17                   5
3  2017-18                   4
4  2018-19                   6
5  2019-20                   0
6  2020-21                   0
7  2021-22                   6
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 694


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0041400406,10,201939,Stephen Curry,1610612744,Golden State Warriors,1,11,14,...,Center(C),16-24 ft.,20.0,-27.0,208.0,1,0,20150616,CLE,GSW
1,Shot Chart Detail,0041400406,25,201939,Stephen Curry,1610612744,Golden State Warriors,1,8,43,...,Center(C),Less Than 8 ft.,0.0,2.0,1.0,1,1,20150616,CLE,GSW
2,Shot Chart Detail,0041400406,27,201939,Stephen Curry,1610612744,Golden State Warriors,1,8,9,...,Left Side Center(LC),24+ ft.,25.0,-179.0,187.0,1,0,20150616,CLE,GSW
3,Shot Chart Detail,0041400406,50,201939,Stephen Curry,1610612744,Golden State Warriors,1,6,0,...,Center(C),Less Than 8 ft.,0.0,9.0,-3.0,1,1,20150616,CLE,GSW
4,Shot Chart Detail,0041400406,56,201939,Stephen Curry,1610612744,Golden State Warriors,1,5,15,...,Left Side(L),24+ ft.,22.0,-228.0,-3.0,1,1,20150616,CLE,GSW
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
689,Shot Chart Detail,0042100401,538,201939,Stephen Curry,1610612744,Golden State Warriors,4,6,47,...,Center(C),16-24 ft.,19.0,60.0,186.0,1,1,20220602,GSW,BOS
690,Shot Chart Detail,0042100401,549,201939,Stephen Curry,1610612744,Golden State Warriors,4,6,5,...,Center(C),Less Than 8 ft.,6.0,51.0,43.0,1,1,20220602,GSW,BOS
691,Shot Chart Detail,0042100401,556,201939,Stephen Curry,1610612744,Golden State Warriors,4,5,23,...,Center(C),8-16 ft.,10.0,13.0,104.0,1,0,20220602,GSW,BOS
692,Shot Chart Detail,0042100401,571,201939,Stephen Curry,1610612744,Golden State Warriors,4,3,55,...,Center(C),24+ ft.,27.0,32.0,273.0,1,0,20220602,GSW,BOS


# Kevin Durant - Golden State Warriors (2017, 2018, 2019)

In [9]:
shots_df, finals_summary = finals_shots_last_10_seasons("Kevin Durant", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   5
3  2017-18                   4
4  2018-19                   1
5  2019-20                   0
6  2020-21                   0
7  2021-22                   0
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 189


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0041600405,7,201142,Kevin Durant,1610612744,Golden State Warriors,1,11,22,...,Center(C),Less Than 8 ft.,1,-11,11,1,1,20170612,GSW,CLE
1,Shot Chart Detail,0041600405,25,201142,Kevin Durant,1610612744,Golden State Warriors,1,9,12,...,Center(C),8-16 ft.,12,-37,116,1,0,20170612,GSW,CLE
2,Shot Chart Detail,0041600405,41,201142,Kevin Durant,1610612744,Golden State Warriors,1,8,26,...,Center(C),Less Than 8 ft.,1,9,16,1,1,20170612,GSW,CLE
3,Shot Chart Detail,0041600405,62,201142,Kevin Durant,1610612744,Golden State Warriors,1,6,37,...,Left Side Center(LC),24+ ft.,25,-201,164,1,0,20170612,GSW,CLE
4,Shot Chart Detail,0041600405,80,201142,Kevin Durant,1610612744,Golden State Warriors,1,5,24,...,Left Side(L),8-16 ft.,11,-117,11,1,1,20170612,GSW,CLE


# Giannis Antetokounmpo - Milwaukee Bucks (2020)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Giannis Antetokounmpo", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   0
3  2017-18                   0
4  2018-19                   0
5  2019-20                   0
6  2020-21                   6
7  2021-22                   0
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 123


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042000406,15,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,11,6,...,Center(C),Less Than 8 ft.,3,27,24,1,1,20210720,MIL,PHX
1,Shot Chart Detail,0042000406,67,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,5,57,...,Left Side Center(LC),24+ ft.,27,-181,211,1,0,20210720,MIL,PHX
2,Shot Chart Detail,0042000406,137,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,1,42,...,Center(C),Less Than 8 ft.,2,-14,17,1,1,20210720,MIL,PHX
3,Shot Chart Detail,0042000406,140,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,1,13,...,Center(C),Less Than 8 ft.,1,6,12,1,1,20210720,MIL,PHX
4,Shot Chart Detail,0042000406,149,203507,Giannis Antetokounmpo,1610612749,Milwaukee Bucks,1,0,13,...,Center(C),Less Than 8 ft.,3,-29,14,1,0,20210720,MIL,PHX


# Nikola Jokic - Denver Nuggets (2023)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Nikola Jokic", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   0
3  2017-18                   0
4  2018-19                   0
5  2019-20                   0
6  2020-21                   0
7  2021-22                   0
8  2022-23                   5
9  2023-24                   0
TOTAL SHOTS: 96


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042200405,83,203999,Nikola Jokić,1610612743,Denver Nuggets,1,5,28,...,Right Side Center(RC),24+ ft.,25,86,243,1,1,20230612,DEN,MIA
1,Shot Chart Detail,0042200405,237,203999,Nikola Jokić,1610612743,Denver Nuggets,2,8,4,...,Center(C),Less Than 8 ft.,5,-5,50,1,1,20230612,DEN,MIA
2,Shot Chart Detail,0042200405,258,203999,Nikola Jokić,1610612743,Denver Nuggets,2,6,56,...,Center(C),Less Than 8 ft.,2,-16,22,1,1,20230612,DEN,MIA
3,Shot Chart Detail,0042200405,274,203999,Nikola Jokić,1610612743,Denver Nuggets,2,5,0,...,Center(C),Less Than 8 ft.,7,-43,62,1,1,20230612,DEN,MIA
4,Shot Chart Detail,0042200405,295,203999,Nikola Jokić,1610612743,Denver Nuggets,2,2,44,...,Center(C),24+ ft.,26,-11,260,1,0,20230612,DEN,MIA


# Jimmy Butler - Miami Heat (2020, 2023)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Jimmy Butler", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2015-16                   0
1  2016-17                   0
2  2017-18                   0
3  2018-19                   0
4  2019-20                   6
5  2020-21                   0
6  2021-22                   0
7  2022-23                   5
8  2023-24                   0
TOTAL SHOTS: 188


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0041900406,49,202710,Jimmy Butler III,1610612748,Miami Heat,1,7,56,...,Center(C),Less Than 8 ft.,6,-7,62,1,0,20201011,MIA,LAL
1,Shot Chart Detail,0041900406,107,202710,Jimmy Butler III,1610612748,Miami Heat,1,2,31,...,Left Side(L),24+ ft.,24,-234,79,1,0,20201011,MIA,LAL
2,Shot Chart Detail,0041900406,115,202710,Jimmy Butler III,1610612748,Miami Heat,1,1,38,...,Right Side(R),24+ ft.,22,226,8,1,1,20201011,MIA,LAL
3,Shot Chart Detail,0041900406,177,202710,Jimmy Butler III,1610612748,Miami Heat,2,10,16,...,Center(C),Less Than 8 ft.,0,0,-6,1,0,20201011,MIA,LAL
4,Shot Chart Detail,0041900406,212,202710,Jimmy Butler III,1610612748,Miami Heat,2,7,57,...,Center(C),8-16 ft.,12,18,120,1,1,20201011,MIA,LAL


# High-Usage Shooters 
### (Klay Thompson, Jayson Tatum, Devin Booker, Jamal Murray)

# Klay Thompson - Golden State Warriors (2015, 2016, 2017, 2018, 2019, 2022)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Klay Thompson", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   6
1  2015-16                   7
2  2016-17                   5
3  2017-18                   4
4  2018-19                   5
5  2019-20                   0
6  2020-21                   0
7  2021-22                   6
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 504


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0041400406,14,202691,Klay Thompson,1610612744,Golden State Warriors,1,10,25,...,Left Side Center(LC),16-24 ft.,17.0,-120.0,132.0,1,1,20150616,CLE,GSW
1,Shot Chart Detail,0041400406,116,202691,Klay Thompson,1610612744,Golden State Warriors,2,11,33,...,Left Side(L),8-16 ft.,14.0,-89.0,112.0,1,0,20150616,CLE,GSW
2,Shot Chart Detail,0041400406,127,202691,Klay Thompson,1610612744,Golden State Warriors,2,10,19,...,Left Side Center(LC),24+ ft.,25.0,-224.0,123.0,1,0,20150616,CLE,GSW
3,Shot Chart Detail,0041400406,420,202691,Klay Thompson,1610612744,Golden State Warriors,3,0,37,...,Left Side Center(LC),24+ ft.,24.0,-168.0,179.0,1,0,20150616,CLE,GSW
4,Shot Chart Detail,0041400406,453,202691,Klay Thompson,1610612744,Golden State Warriors,4,10,39,...,Center(C),8-16 ft.,11.0,-8.0,113.0,1,0,20150616,CLE,GSW


# Jayson Tatum - Boston Celtics (2022, 2024)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Jayson Tatum", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2015-16                   0
1  2016-17                   0
2  2017-18                   0
3  2018-19                   0
4  2019-20                   0
5  2020-21                   0
6  2021-22                   6
7  2022-23                   0
8  2023-24                   5
TOTAL SHOTS: 218


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042100406,20,1628369,Jayson Tatum,1610612738,Boston Celtics,1,10,34,...,Center(C),Less Than 8 ft.,4,-45,21,1,0,20220616,BOS,GSW
1,Shot Chart Detail,0042100406,31,1628369,Jayson Tatum,1610612738,Boston Celtics,1,9,27,...,Left Side Center(LC),24+ ft.,25,-130,217,1,1,20220616,BOS,GSW
2,Shot Chart Detail,0042100406,53,1628369,Jayson Tatum,1610612738,Boston Celtics,1,8,3,...,Left Side Center(LC),16-24 ft.,17,-93,154,1,1,20220616,BOS,GSW
3,Shot Chart Detail,0042100406,66,1628369,Jayson Tatum,1610612738,Boston Celtics,1,6,27,...,Left Side Center(LC),24+ ft.,25,-92,243,1,0,20220616,BOS,GSW
4,Shot Chart Detail,0042100406,120,1628369,Jayson Tatum,1610612738,Boston Celtics,1,2,53,...,Left Side Center(LC),16-24 ft.,16,-61,152,1,1,20220616,BOS,GSW


# Devin Booker - Pheonix Suns (2021)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Devin Booker", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2015-16                   0
1  2016-17                   0
2  2017-18                   0
3  2018-19                   0
4  2019-20                   0
5  2020-21                   6
6  2021-22                   0
7  2022-23                   0
8  2023-24                   0
TOTAL SHOTS: 143


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042000406,17,1626164,Devin Booker,1610612756,Phoenix Suns,1,10,48,...,Right Side(R),8-16 ft.,14,84,121,1,1,20210720,MIL,PHX
1,Shot Chart Detail,0042000406,21,1626164,Devin Booker,1610612756,Phoenix Suns,1,10,13,...,Right Side Center(RC),24+ ft.,26,117,239,1,0,20210720,MIL,PHX
2,Shot Chart Detail,0042000406,50,1626164,Devin Booker,1610612756,Phoenix Suns,1,8,1,...,Left Side(L),8-16 ft.,12,-123,17,1,0,20210720,MIL,PHX
3,Shot Chart Detail,0042000406,108,1626164,Devin Booker,1610612756,Phoenix Suns,1,3,22,...,Center(C),Less Than 8 ft.,3,22,29,1,0,20210720,MIL,PHX
4,Shot Chart Detail,0042000406,123,1626164,Devin Booker,1610612756,Phoenix Suns,1,2,50,...,Center(C),24+ ft.,26,57,260,1,0,20210720,MIL,PHX


# Jamal Murray - Denver Nuggets (2023)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Jamal Murray", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2015-16                   0
1  2016-17                   0
2  2017-18                   0
3  2018-19                   0
4  2019-20                   0
5  2020-21                   0
6  2021-22                   0
7  2022-23                   5
8  2023-24                   0
TOTAL SHOTS: 91


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042200405,50,1627750,Jamal Murray,1610612743,Denver Nuggets,1,7,15,...,Center(C),24+ ft.,25,56,253,1,0,20230612,DEN,MIA
1,Shot Chart Detail,0042200405,65,1627750,Jamal Murray,1610612743,Denver Nuggets,1,6,47,...,Center(C),Less Than 8 ft.,1,1,16,1,1,20230612,DEN,MIA
2,Shot Chart Detail,0042200405,146,1627750,Jamal Murray,1610612743,Denver Nuggets,1,1,34,...,Left Side Center(LC),24+ ft.,25,-137,220,1,0,20230612,DEN,MIA
3,Shot Chart Detail,0042200405,158,1627750,Jamal Murray,1610612743,Denver Nuggets,1,1,11,...,Left Side Center(LC),24+ ft.,25,-170,196,1,0,20230612,DEN,MIA
4,Shot Chart Detail,0042200405,171,1627750,Jamal Murray,1610612743,Denver Nuggets,1,0,1,...,Left Side Center(LC),16-24 ft.,20,-137,149,1,0,20230612,DEN,MIA


# Low-Usage Role Players
### (Aaron Gordon, Derrick White)

## Aaron Gordon - Denver Nuggets (2022-23)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Aaron Gordon", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   0
3  2017-18                   0
4  2018-19                   0
5  2019-20                   0
6  2020-21                   0
7  2021-22                   0
8  2022-23                   5
9  2023-24                   0
TOTAL SHOTS: 48


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042200405,13,203932,Aaron Gordon,1610612743,Denver Nuggets,1,10,58,...,Center(C),Less Than 8 ft.,3,-14,27,1,0,20230612,DEN,MIA
1,Shot Chart Detail,0042200405,47,203932,Aaron Gordon,1610612743,Denver Nuggets,1,7,48,...,Center(C),8-16 ft.,10,23,102,1,1,20230612,DEN,MIA
2,Shot Chart Detail,0042200405,357,203932,Aaron Gordon,1610612743,Denver Nuggets,3,10,54,...,Center(C),24+ ft.,25,67,246,1,0,20230612,DEN,MIA
3,Shot Chart Detail,0042200405,420,203932,Aaron Gordon,1610612743,Denver Nuggets,3,5,2,...,Center(C),Less Than 8 ft.,1,10,6,1,0,20230612,DEN,MIA
4,Shot Chart Detail,0042200405,547,203932,Aaron Gordon,1610612743,Denver Nuggets,4,7,48,...,Left Side(L),24+ ft.,22,-223,2,1,0,20230612,DEN,MIA


## Derrick White - Boston Celtics (2022, 2024)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Derrick White", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   0
3  2017-18                   0
4  2018-19                   0
5  2019-20                   0
6  2020-21                   0
7  2021-22                   6
8  2022-23                   0
9  2023-24                   5
TOTAL SHOTS: 109


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042100406,72,1628401,Derrick White,1610612738,Boston Celtics,1,5,57,...,Center(C),Less Than 8 ft.,0,7,4,1,1,20220616,BOS,GSW
1,Shot Chart Detail,0042100406,109,1628401,Derrick White,1610612738,Boston Celtics,1,3,11,...,Center(C),Less Than 8 ft.,4,-21,37,1,0,20220616,BOS,GSW
2,Shot Chart Detail,0042100406,139,1628401,Derrick White,1610612738,Boston Celtics,1,1,6,...,Left Side Center(LC),24+ ft.,26,-121,235,1,0,20220616,BOS,GSW
3,Shot Chart Detail,0042100406,143,1628401,Derrick White,1610612738,Boston Celtics,1,0,33,...,Right Side(R),8-16 ft.,8,81,-8,1,0,20220616,BOS,GSW
4,Shot Chart Detail,0042100406,166,1628401,Derrick White,1610612738,Boston Celtics,2,11,17,...,Center(C),Less Than 8 ft.,3,-38,11,1,0,20220616,BOS,GSW


## Corner Specialists 
### (P.J.Tucker, Danny Green, Grant Williams, Kentavious-pope)

## P.J. Tucker - Milwaukee Bucks (2020)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("P.J. Tucker", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   0
3  2017-18                   0
4  2018-19                   0
5  2019-20                   0
6  2020-21                   6
7  2021-22                   0
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 20


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042000406,240,200782,P.J. Tucker,1610612749,Milwaukee Bucks,2,5,12,...,Left Side(L),24+ ft.,23,-231,1,1,0,20210720,MIL,PHX
1,Shot Chart Detail,0042000405,281,200782,P.J. Tucker,1610612749,Milwaukee Bucks,2,3,13,...,Right Side(R),24+ ft.,23,227,44,1,1,20210717,PHX,MIL
2,Shot Chart Detail,0042000405,481,200782,P.J. Tucker,1610612749,Milwaukee Bucks,4,10,19,...,Right Side(R),8-16 ft.,12,110,64,1,0,20210717,PHX,MIL
3,Shot Chart Detail,0042000404,514,200782,P.J. Tucker,1610612749,Milwaukee Bucks,4,9,16,...,Right Side(R),24+ ft.,22,229,17,1,0,20210714,MIL,PHX
4,Shot Chart Detail,0042000403,264,200782,P.J. Tucker,1610612749,Milwaukee Bucks,2,4,6,...,Center(C),Less Than 8 ft.,2,25,13,1,1,20210711,MIL,PHX


## Danny Green - Toronto Raptors and Los Angeles Lakers (2019, 2020)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Danny Green", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   0
3  2017-18                   0
4  2018-19                   6
5  2019-20                   6
6  2020-21                   0
7  2021-22                   0
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 90


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0041800405,46,201980,Danny Green,1610612761,Toronto Raptors,1,8,16,...,Left Side(L),16-24 ft.,20,-209,-13,1,1,20190610,TOR,GSW
1,Shot Chart Detail,0041800405,69,201980,Danny Green,1610612761,Toronto Raptors,1,6,23,...,Center(C),Less Than 8 ft.,0,6,4,1,1,20190610,TOR,GSW
2,Shot Chart Detail,0041800405,331,201980,Danny Green,1610612761,Toronto Raptors,2,2,42,...,Left Side(L),24+ ft.,23,-230,6,1,0,20190610,TOR,GSW
3,Shot Chart Detail,0041800405,365,201980,Danny Green,1610612761,Toronto Raptors,2,0,0,...,Right Side Center(RC),24+ ft.,24,187,159,1,0,20190610,TOR,GSW
4,Shot Chart Detail,0041800405,467,201980,Danny Green,1610612761,Toronto Raptors,3,3,6,...,Center(C),Less Than 8 ft.,0,0,-6,1,0,20190610,TOR,GSW


## Grant Williams - Boston Celtics (2022)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Grant Williams", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   0
3  2017-18                   0
4  2018-19                   0
5  2019-20                   0
6  2020-21                   0
7  2021-22                   6
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 17


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042100406,287,1629684,Grant Williams,1610612738,Boston Celtics,2,3,25,...,Left Side Center(LC),24+ ft.,25,-140,215,1,0,20220616,BOS,GSW
1,Shot Chart Detail,0042100406,587,1629684,Grant Williams,1610612738,Boston Celtics,4,6,37,...,Center(C),Less Than 8 ft.,3,10,36,1,1,20220616,BOS,GSW
2,Shot Chart Detail,0042100405,175,1629684,Grant Williams,1610612738,Boston Celtics,2,10,23,...,Right Side Center(RC),24+ ft.,27,148,232,1,0,20220613,GSW,BOS
3,Shot Chart Detail,0042100405,419,1629684,Grant Williams,1610612738,Boston Celtics,3,3,55,...,Center(C),Less Than 8 ft.,2,-22,19,1,1,20220613,GSW,BOS
4,Shot Chart Detail,0042100404,103,1629684,Grant Williams,1610612738,Boston Celtics,1,3,57,...,Left Side(L),24+ ft.,23,-228,73,1,0,20220610,BOS,GSW


## Ketavious Caldwell-Pope - Los Angeles Lakers and Denver Nuggets (2020,2023)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Kentavious Caldwell-Pope", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   0
3  2017-18                   0
4  2018-19                   0
5  2019-20                   6
6  2020-21                   0
7  2021-22                   0
8  2022-23                   5
9  2023-24                   0
TOTAL SHOTS: 100


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0041900406,15,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,1,10,53,...,Center(C),16-24 ft.,19,-14,194,1,0,20201011,MIA,LAL
1,Shot Chart Detail,0041900406,77,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,1,4,58,...,Center(C),24+ ft.,25,-11,253,1,0,20201011,MIA,LAL
2,Shot Chart Detail,0041900406,85,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,1,4,12,...,Center(C),Less Than 8 ft.,1,-15,13,1,1,20201011,MIA,LAL
3,Shot Chart Detail,0041900406,101,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,1,3,2,...,Right Side(R),24+ ft.,23,235,48,1,0,20201011,MIA,LAL
4,Shot Chart Detail,0041900406,266,203484,Kentavious Caldwell-Pope,1610612747,Los Angeles Lakers,2,4,19,...,Left Side(L),8-16 ft.,11,-107,26,1,1,20201011,MIA,LAL


## Defensive-First Players
### (Draymond Green, Marcus Smart, Jaylen Brown, Bam Adebayo)

## Draymond Green - Golden State Warriors (2014-2019, 2021)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Draymond Green", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   6
1  2015-16                   6
2  2016-17                   5
3  2017-18                   4
4  2018-19                   6
5  2019-20                   0
6  2020-21                   0
7  2021-22                   6
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 319


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0041400406,2,203110,Draymond Green,1610612744,Golden State Warriors,1,11,41,...,Right Side(R),16-24 ft.,21,214,14,1,0,20150616,CLE,GSW
1,Shot Chart Detail,0041400406,31,203110,Draymond Green,1610612744,Golden State Warriors,1,7,50,...,Center(C),Less Than 8 ft.,7,50,50,1,1,20150616,CLE,GSW
2,Shot Chart Detail,0041400406,78,203110,Draymond Green,1610612744,Golden State Warriors,1,2,33,...,Center(C),24+ ft.,24,18,247,1,1,20150616,CLE,GSW
3,Shot Chart Detail,0041400406,154,203110,Draymond Green,1610612744,Golden State Warriors,2,8,48,...,Left Side Center(LC),24+ ft.,24,-150,195,1,0,20150616,CLE,GSW
4,Shot Chart Detail,0041400406,193,203110,Draymond Green,1610612744,Golden State Warriors,2,6,13,...,Center(C),Less Than 8 ft.,7,-56,42,1,1,20150616,CLE,GSW


## Marcus Smart - Boston Celtics (2021)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Marcus Smart", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   0
3  2017-18                   0
4  2018-19                   0
5  2019-20                   0
6  2020-21                   0
7  2021-22                   6
8  2022-23                   0
9  2023-24                   0
TOTAL SHOTS: 79


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042100406,13,203935,Marcus Smart,1610612738,Boston Celtics,1,11,11,...,Center(C),8-16 ft.,14,-59,132,1,0,20220616,BOS,GSW
1,Shot Chart Detail,0042100406,23,203935,Marcus Smart,1610612738,Boston Celtics,1,10,24,...,Center(C),8-16 ft.,10,-20,99,1,1,20220616,BOS,GSW
2,Shot Chart Detail,0042100406,362,203935,Marcus Smart,1610612738,Boston Celtics,3,10,22,...,Center(C),Less Than 8 ft.,4,41,25,1,0,20220616,BOS,GSW
3,Shot Chart Detail,0042100406,386,203935,Marcus Smart,1610612738,Boston Celtics,3,7,26,...,Left Side(L),8-16 ft.,12,-97,73,1,0,20220616,BOS,GSW
4,Shot Chart Detail,0042100406,394,203935,Marcus Smart,1610612738,Boston Celtics,3,6,22,...,Left Side Center(LC),24+ ft.,26,-147,225,1,0,20220616,BOS,GSW


## Jaylen Brown - Boston Celtics (2021, 2024)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Jaylen Brown", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   0
3  2017-18                   0
4  2018-19                   0
5  2019-20                   0
6  2020-21                   0
7  2021-22                   6
8  2022-23                   0
9  2023-24                   5
TOTAL SHOTS: 200


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0042100406,17,1627759,Jaylen Brown,1610612738,Boston Celtics,1,10,52,...,Left Side Center(LC),16-24 ft.,18,-114,148,1,1,20220616,BOS,GSW
1,Shot Chart Detail,0042100406,27,1627759,Jaylen Brown,1610612738,Boston Celtics,1,9,55,...,Center(C),24+ ft.,27,-61,273,1,1,20220616,BOS,GSW
2,Shot Chart Detail,0042100406,56,1627759,Jaylen Brown,1610612738,Boston Celtics,1,7,32,...,Left Side Center(LC),24+ ft.,25,-163,197,1,0,20220616,BOS,GSW
3,Shot Chart Detail,0042100406,60,1627759,Jaylen Brown,1610612738,Boston Celtics,1,6,56,...,Center(C),Less Than 8 ft.,5,52,12,1,0,20220616,BOS,GSW
4,Shot Chart Detail,0042100406,105,1627759,Jaylen Brown,1610612738,Boston Celtics,1,3,31,...,Right Side(R),8-16 ft.,14,142,11,1,1,20220616,BOS,GSW


## Bam Adebayo - Miami Heat (2019,2022)

In [ ]:
shots_df, finals_summary = finals_shots_last_10_seasons("Bam Adebayo", 2014, 2024)
print(finals_summary)
print("TOTAL SHOTS:", len(shots_df))
shots_df.head()

    season  finals_games_found
0  2014-15                   0
1  2015-16                   0
2  2016-17                   0
3  2017-18                   0
4  2018-19                   0
5  2019-20                   4
6  2020-21                   0
7  2021-22                   0
8  2022-23                   5
9  2023-24                   0
TOTAL SHOTS: 142


,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0041900406,9,1628389,Bam Adebayo,1610612748,Miami Heat,1,11,25,...,Center(C),Less Than 8 ft.,1,-6,12,1,1,20201011,MIA,LAL
1,Shot Chart Detail,0041900406,28,1628389,Bam Adebayo,1610612748,Miami Heat,1,9,28,...,Center(C),Less Than 8 ft.,2,-22,17,1,0,20201011,MIA,LAL
2,Shot Chart Detail,0041900406,59,1628389,Bam Adebayo,1610612748,Miami Heat,1,6,27,...,Center(C),8-16 ft.,10,-5,102,1,0,20201011,MIA,LAL
3,Shot Chart Detail,0041900406,160,1628389,Bam Adebayo,1610612748,Miami Heat,2,11,44,...,Center(C),Less Than 8 ft.,2,-3,20,1,1,20201011,MIA,LAL
4,Shot Chart Detail,0041900406,231,1628389,Bam Adebayo,1610612748,Miami Heat,2,6,28,...,Center(C),Less Than 8 ft.,6,65,23,1,0,20201011,MIA,LAL
